In [1]:
from datasets import load_dataset
from utils import utils
import re
from typing import List, Dict, Tuple
import hashlib
from collections import Counter

In [2]:
lean_solver_reasoning_dataset = utils.read_from_json("./data/collected_Math-500.json")
NL_solver_reasoning_dataset = utils.read_from_json("./data/NL_answered_Math-500.json")

In [12]:
lean_solver_reasoning_dict = {curr["Name"]: curr for curr in lean_solver_reasoning_dataset}

In [15]:
lean_solver_reasoning_dict["math500_intermediate_algebra_1994"].keys()

dict_keys(['problem', 'solution', 'answer', 'subject', 'level', 'unique_id', 'Name', 'translation_generated_results', 'autoformalization_generated_results', 'generated_proof_results', 'collected_answer_results'])

In [46]:
print(lean_solver_reasoning_dict["math500_precalculus_675"]["autoformalization_generated_results"][0]["autoformalization_generated_content"])

theorem math500_precalculus_675_2 :
  ∃ (C : ℝ), IsLeast {C | 0 < C ∧ ∀ (v : EuclideanSpace ℝ (Fin 2)), ‖(2 * v 0 + 3 * v 1, -2 * v 0)‖ ≤ C * ‖v‖} C := by sorry


In [60]:
print(lean_solver_reasoning_dict["math500_precalculus_675"]["generated_proof_results"][0]["response_proof_generation"])

<|im_start|>system
You are an expert in mathematics and Lean 4.<|im_end|>
<|im_start|>user
Think about and solve the following problem step by step in Lean 4.
# Problem: Find the smallest positive real number $C$ for which
\[\left\| \begin{pmatrix} 2 & 3 \\ 0 & -2 \end{pmatrix} \bold{v} \right\| \le C \|\bold{v}\|\]for all two-dimensional vectors $\bold{v}.$

Note that for a two-dimensional vector $\mathbf{a},$ $\|\mathbf{a}\|$ is the magnitude of $\mathbf{a}.$
# Formal statement:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Find the smallest positive real number $C$ for which
\[\left\| \begin{pmatrix} 2 & 3 \\ 0 & -2 \end{pmatrix} \bold{v} \right\| \le C \|\bold{v}\|\]for all two-dimensional vectors $\bold{v}.$

Note that for a two-dimensional vector $\mathbf{a},$ $\|\mathbf{a}\|$ is the magnitude of $\mathbf{a}.$ -/
theorem math500_precalculus_675_2 :
  ∃ (C : ℝ), IsLeast {C | 0 < C ∧ ∀ (v : EuclideanSpace ℝ (Fin 2)), 

In [49]:
print(lean_solver_reasoning_dict["math500_precalculus_675"]["translation_generated_results"][2]["translated_existence_problem"])

Prove that there exists a smallest positive real number $C$ such that
$$\left\| \begin{pmatrix} 2 & 3 \\ 0 & -2 \end{pmatrix} \bold{v} \right\| \le C \|\bold{v}\|$$
for all two-dimensional vectors $\bold{v}.$


In [48]:
print(lean_solver_reasoning_dict["math500_precalculus_675"]["problem"])

Find the smallest positive real number $C$ for which
\[\left\| \begin{pmatrix} 2 & 3 \\ 0 & -2 \end{pmatrix} \bold{v} \right\| \le C \|\bold{v}\|\]for all two-dimensional vectors $\bold{v}.$

Note that for a two-dimensional vector $\mathbf{a},$ $\|\mathbf{a}\|$ is the magnitude of $\mathbf{a}.$


In [50]:
print(lean_solver_reasoning_dict["math500_precalculus_675"]["answer"])

4


In [4]:
total_question_num = len(lean_solver_reasoning_dataset)
total_pass_correct_num = 0
total_majority_correct_num = 0

wrong_answer_idx = []

answer_data_total = {}

for i, curr in enumerate(lean_solver_reasoning_dataset):
    curr_answers = utils.collect_answers(curr["collected_answer_results"], answer_key="collected_answer")
    pass_correceness, majority_vote_correctness = utils.judge_correctness(curr_answers, curr["answer"])
    total_pass_correct_num += pass_correceness
    total_majority_correct_num += majority_vote_correctness
    curr_answer_record = {
        "Name": curr["Name"],
        "Problem": curr["problem"],
        "Answer": curr["answer"],
        "Subject": curr["subject"],
        "FL_answers": curr_answers
    }
    answer_data_total[curr["Name"]] = curr_answer_record
    if not pass_correceness:
        wrong_answer_idx.append(i)
print(f"Total number of questions: {total_question_num}")
print(f"Total number of questions passed: {total_pass_correct_num}")
print(f"Total number of questions majority voted: {total_majority_correct_num}")
print(f"Pass rate: {total_pass_correct_num / total_question_num}")
print(f"Majority vote rate: {total_majority_correct_num / total_question_num}")

Total number of questions: 500
Total number of questions passed: 429
Total number of questions majority voted: 395
Pass rate: 0.858
Majority vote rate: 0.79


In [5]:
total_question_num = len(NL_solver_reasoning_dataset)
total_pass_correct_num_NL = 0
total_majority_correct_num_NL = 0

wrong_answer_idx_NL = []

for i, curr in enumerate(NL_solver_reasoning_dataset):
    curr_answers = utils.collect_answers(curr["solved_answer_results"], answer_key="solved_answer")
    pass_correceness, majority_vote_correctness = utils.judge_correctness(curr_answers[:int(len(curr_answers)/2)], curr["answer"])
    total_pass_correct_num_NL += pass_correceness
    total_majority_correct_num_NL += majority_vote_correctness
    answer_data_total[curr['Name']]["NL_answers"] = curr_answers
    if not pass_correceness:
        wrong_answer_idx_NL.append(i)
print(f"Total number of questions: {total_question_num}")
print(f"Total number of questions passed: {total_pass_correct_num_NL}")
print(f"Total number of questions majority voted: {total_majority_correct_num_NL}")
print(f"Pass rate: {total_pass_correct_num_NL / total_question_num}")
print(f"Majority vote rate: {total_majority_correct_num_NL / total_question_num}")

Total number of questions: 500
Total number of questions passed: 423
Total number of questions majority voted: 416
Pass rate: 0.846
Majority vote rate: 0.832


In [6]:
answer_data_total = list(answer_data_total.values())
for curr in answer_data_total:
    curr["Hybrid_answers"] = curr["FL_answers"] + curr["NL_answers"][:int(len(curr["NL_answers"])/2)]

In [7]:
utils.write_to_json("./data/Math-500_answer_storage.json", answer_data_total)

In [3]:
answer_data_total = utils.read_from_json("./data/Math-500_answer_storage.json")

In [5]:
subject_set = set()
for curr in answer_data_total:
    subject_set.add(curr["Subject"])
subject_ls = list(subject_set)

In [9]:
results_by_subject = {}
for subject in subject_ls:
    results_by_subject[subject] = {
        "total_question_num": 0,
        "total_pass_correct_num_NL": 0,
        "total_pass_correct_num_hybrid": 0,
    }

for curr in answer_data_total:
    curr_subject = curr["Subject"]
    pass_correceness_hybrid, majority_vote_correctness_hybrid = utils.judge_correctness(curr["Hybrid_answers"], curr["Answer"])
    pass_correceness_NL, majority_vote_correctness_NL = utils.judge_correctness(curr["NL_answers"], curr["Answer"])
    results_by_subject[curr_subject]["total_question_num"] += 1
    results_by_subject[curr_subject]["total_pass_correct_num_hybrid"] += pass_correceness_hybrid
    results_by_subject[curr_subject]["total_pass_correct_num_NL"] += pass_correceness_NL
    

In [15]:
for curr_subject in results_by_subject:
    curr_subject_result = results_by_subject[curr_subject]
    curr_subject_result["pass_rate_hybrid"] = curr_subject_result["total_pass_correct_num_hybrid"] / curr_subject_result["total_question_num"]
    curr_subject_result["pass_rate_NL"] = curr_subject_result["total_pass_correct_num_NL"] / curr_subject_result["total_question_num"]
    print(f"Subject: {curr_subject}")
    print(f"Total number of questions: {curr_subject_result['total_question_num']}")
    print(f"Total number of questions passed (Hybrid): {curr_subject_result['total_pass_correct_num_hybrid']}")
    print(f"Total number of questions passed (NL): {curr_subject_result['total_pass_correct_num_NL']}")
    print(f"Pass rate (Hybrid): {(curr_subject_result['pass_rate_hybrid'] * 100):.2f}\%")
    print(f"Pass rate (NL): {(curr_subject_result['pass_rate_NL'] * 100):.2f}\%")
    print(f"Improvements: {((curr_subject_result['pass_rate_hybrid'] - curr_subject_result['pass_rate_NL']) * 100):.2f}\%")
    print("-" * 50)

Subject: Prealgebra
Total number of questions: 82
Total number of questions passed (Hybrid): 73
Total number of questions passed (NL): 69
Pass rate (Hybrid): 89.02\%
Pass rate (NL): 84.15\%
Improvements: 4.88\%
--------------------------------------------------
Subject: Counting & Probability
Total number of questions: 38
Total number of questions passed (Hybrid): 34
Total number of questions passed (NL): 32
Pass rate (Hybrid): 89.47\%
Pass rate (NL): 84.21\%
Improvements: 5.26\%
--------------------------------------------------
Subject: Intermediate Algebra
Total number of questions: 97
Total number of questions passed (Hybrid): 87
Total number of questions passed (NL): 82
Pass rate (Hybrid): 89.69\%
Pass rate (NL): 84.54\%
Improvements: 5.15\%
--------------------------------------------------
Subject: Geometry
Total number of questions: 41
Total number of questions passed (Hybrid): 33
Total number of questions passed (NL): 27
Pass rate (Hybrid): 80.49\%
Pass rate (NL): 65.85\%
Impr

In [3]:
total_pass_correct_num_hybrid = 0
total_majority_correct_num_hybrid = 0
total_pass_correct_num_FL = 0
total_majority_correct_num_FL = 0
total_pass_correct_num_NL = 0
total_majority_correct_num_NL = 0
total_pass_correct_num_NL_pass8 = 0
total_majority_correct_num_NL_pass8 = 0

total_pass_correct_num_FL_pass1 = 0
total_majority_correct_num_FL_pass1 = 0
total_pass_correct_num_NL_pass1 = 0
total_majority_correct_num_NL_pass1 = 0

total_question_num_hybrid = 0
wrong_answer_idx_hybrid = []
wrong_answer_idx_FL = []
wrong_answer_idx_NL = []
FL_correct_NL_wrong_ls = []

for i, curr in enumerate(answer_data_total):
    # if curr["Subject"] not in valid_tag_set:
    #     continue
    total_question_num_hybrid += 1

    pass_correceness_hybrid, majority_vote_correctness_hybrid = utils.judge_correctness(curr["Hybrid_answers"], curr["Answer"])
    total_pass_correct_num_hybrid += pass_correceness_hybrid
    total_majority_correct_num_hybrid += majority_vote_correctness_hybrid

    pass_correceness_FL, majority_vote_correctness_FL = utils.judge_correctness(curr["FL_answers"], curr["Answer"])
    total_pass_correct_num_FL += pass_correceness_FL
    total_majority_correct_num_FL += majority_vote_correctness_FL

    pass_correceness_NL, majority_vote_correctness_NL = utils.judge_correctness(curr["NL_answers"], curr["Answer"])
    total_pass_correct_num_NL += pass_correceness_NL
    total_majority_correct_num_NL += majority_vote_correctness_NL

    pass_correceness_NL_pass8, majority_vote_correctness_NL_pass8 = utils.judge_correctness(curr["NL_answers"][:int(len(curr["NL_answers"])/2)], curr["Answer"])
    total_pass_correct_num_NL_pass8 += pass_correceness_NL_pass8
    total_majority_correct_num_NL_pass8 += majority_vote_correctness_NL_pass8

    pass_correceness_FL_pass1, majority_vote_correctness_FL_pass1 = utils.judge_correctness(curr["FL_answers"][:1], curr["Answer"])
    total_pass_correct_num_FL_pass1 += pass_correceness_FL_pass1
    total_majority_correct_num_FL_pass1 += majority_vote_correctness_FL_pass1
    pass_correceness_NL_pass1, majority_vote_correctness_NL_pass1 = utils.judge_correctness(curr["NL_answers"][:1], curr["Answer"])
    total_pass_correct_num_NL_pass1 += pass_correceness_NL_pass1
    total_majority_correct_num_NL_pass1 += majority_vote_correctness_NL_pass1


    if pass_correceness_FL and not pass_correceness_NL:
        FL_correct_NL_wrong_ls.append(curr)

    if not pass_correceness_hybrid:
        wrong_answer_idx_hybrid.append(i)
    if not pass_correceness_FL:
        wrong_answer_idx_FL.append(i)
    if not pass_correceness_NL:
        wrong_answer_idx_NL.append(i)

print(f"Total number of questions: {total_question_num_hybrid}")
print(f"Pass@16 (Hybrid): {total_pass_correct_num_hybrid / total_question_num_hybrid}")
print(f"Majority vote@16 (Hybrid): {total_majority_correct_num_hybrid / total_question_num_hybrid}")
print("")
print(f"Pass@8 (FL): {total_pass_correct_num_FL / total_question_num_hybrid}")
print(f"Majority vote@8 (FL): {total_majority_correct_num_FL / total_question_num_hybrid}")
print("")
print(f"Pass@8 (NL): {total_pass_correct_num_NL_pass8 / total_question_num_hybrid}")
print(f"Majority vote@8 (NL): {total_majority_correct_num_NL_pass8 / total_question_num_hybrid}")
print("")
print(f"Pass@16 (NL): {total_pass_correct_num_NL / total_question_num_hybrid}")
print(f"Majority vote@16 (NL): {total_majority_correct_num_NL / total_question_num_hybrid}")
print("")
print(f"Pass@1 (FL): {total_pass_correct_num_FL_pass1 / total_question_num_hybrid}")
print(f"Majority vote@1 (FL): {total_majority_correct_num_FL_pass1 / total_question_num_hybrid}")
print("")
print(f"Pass@1 (NL): {total_pass_correct_num_NL_pass1 / total_question_num_hybrid}")
print(f"Majority vote@1 (NL): {total_majority_correct_num_NL_pass1 / total_question_num_hybrid}")


Total number of questions: 500
Pass@16 (Hybrid): 0.898
Majority vote@16 (Hybrid): 0.836

Pass@8 (FL): 0.858
Majority vote@8 (FL): 0.79

Pass@8 (NL): 0.846
Majority vote@8 (NL): 0.832

Pass@16 (NL): 0.852
Majority vote@16 (NL): 0.836

Pass@1 (FL): 0.694
Majority vote@1 (FL): 0.694

Pass@1 (NL): 0.836
Majority vote@1 (NL): 0.836


In [8]:
for curr in FL_correct_NL_wrong_ls:
    curr["problem"] = curr["Problem"]

In [5]:
len(FL_correct_NL_wrong_ls)

23

In [9]:
utils.write_to_json("./data/FL_correct_NL_wrong.json", FL_correct_NL_wrong_ls)

In [10]:
for curr in lean_solver_reasoning_dataset:
    if curr["Name"] == "math500_precalculus_807":
        break

In [11]:
print(curr["generated_proof_results"][1]["response_proof_generation"])

<|im_start|>system
You are an expert in mathematics and Lean 4.<|im_end|>
<|im_start|>user
Think about and solve the following problem step by step in Lean 4.
# Problem: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
# Formal statement:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$ -/
theorem math500_precalculus_807_1 (r : ℝ) (θ : ℝ) (hr : r > 0) (hθ : θ ∈ Ico 0 (2 * π)) :
    (r, θ) = (Real.sqrt (3 ^ 2 + 0 ^ 2), arctan (0 / 3)) := by sorry
```<|im_end|>
<|im_start|>assistant
<think>
# Converting the Point $(0,3)$ from Rectangular to Polar Coordinates

To convert the point $(0,3)$ from rectangular coordinates to polar coordinates, we need to find the

In [12]:
print(curr["translation_generated_results"][1]["translated_existence_problem"])

Prove that the point $(0,3)$ in rectangular coordinates can be uniquely represented in polar coordinates $(r, \theta)$ where $r > 0$ and $0 \le \theta < 2 \pi.$


In [13]:
curr["collected_answer_results"][1]["collected_answer"]

'(3, \\frac{\\pi}{2})'

In [14]:
curr["answer"]

'\\left( 3, \\frac{\\pi}{2} \\right)'